In [1]:
# Setup: Ensure all required packages are available
import sys
import subprocess

def install_if_missing(package_name, import_name=None):
    """Install package if not available."""
    if import_name is None:
        import_name = package_name
    
    try:
        __import__(import_name)
        print(f"✓ {package_name} is available")
    except ImportError:
        print(f"⚠ {package_name} not found. Installing...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])
        print(f"✓ {package_name} installed successfully")

# Check and install critical packages
packages_to_check = [
    ('pandas', 'pandas'),
    ('numpy', 'numpy'),
    ('joblib', 'joblib')
]

print("Checking required packages...")
for pkg, imp in packages_to_check:
    install_if_missing(pkg, imp)

print("\n✅ All required packages are ready!")

Checking required packages...
✓ pandas is available
✓ numpy is available
✓ joblib is available

✅ All required packages are ready!


# Funcao de Predicao - X-Health Credit Risk

Guia pratico de uso da funcao `predict_default()` para inferir probabilidade de default.

**Pipeline do modelo:** preprocessor (ColumnTransformer) -> XGBoost classifier

**Feature engineering automatico:** a funcao `engineer_features()` cria 7 features derivadas
automaticamente a partir dos dados brutos:

| Feature | Formula | Logica |
|---------|---------|--------|
| `total_exposto` | vencido + por_vencer + quitado | Volume total de relacionamento |
| `razao_inadimplencia` | vencido / total_exposto | Grau de deterioracao da carteira |
| `taxa_cobertura_divida` | quitado / pedido | Capacidade historica de pagamento |
| `razao_vencido_pedido` | vencido / pedido | Alavancagem atual |
| `flag_risco_juridico` | 1 se protestos OU acoes judiciais | Sinal binario de alerta |
| `ticket_medio_protestos` | valor_protestos / quant_protestos | Gravidade do problema juridico |
| `qtd_parcelas` | contagem de '/' em forma_pagamento + 1 | Proxy de fluxo de caixa |

**Tratamento de dados:**
- Registros com `valor_total_pedido` negativo sao removidos no treino (possivel inconsistencia nos dados - EDA)
- 6 variaveis com >=99% de zeros foram excluidas do modelo (possivel filtro pre-venda - EDA):
  `participacao_falencia_valor`, `falencia_concordata_qtd`, `acao_judicial_valor`,
  `dividas_vencidas_qtd`, `dividas_vencidas_valor`, `quant_acao_judicial`

**Tratamento de missing values:**
- Categoricas: valores ausentes tratados como `'unknown'` (categoria propria utilizada pelo modelo)
- Numericas: valores ausentes tratados como `NaN` (imputer usa mediana do treino)

**Nota:** A variavel `month` e utilizada como feature categorica (OneHotEncoder).
Correlacao com a taxa de default foi identificada no EDA, indicando sazonalidade.

O pipeline recebe dados brutos e faz feature engineering + preprocessing internamente.

In [2]:
import pandas as pd
import numpy as np
import joblib
import json
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

# Carregar metadata
with open('../models/model_metadata.json', 'r') as f:
    metadata = json.load(f)

# Carregar modelo (pipeline: preprocessor -> classifier)
model_path = list(Path('../models/').glob('model_*.pkl'))[0]
model = joblib.load(model_path)

# Features e thresholds do metadata
ALL_FEATURES = metadata['numeric_features'] + metadata['high_cardinality_features'] + metadata['low_cardinality_features']
CATEGORICAL_FEATURES = metadata['high_cardinality_features'] + metadata['low_cardinality_features']
DERIVED_FEATURES = metadata.get('derived_features', [])
RAW_FEATURES = [f for f in ALL_FEATURES if f not in DERIVED_FEATURES]

THRESHOLD_F1 = metadata.get('optimal_threshold_f1', 0.5)
THRESHOLD_F2 = metadata.get('optimal_threshold_f2', 0.3)
THRESHOLD_F05 = metadata.get('optimal_threshold_f05', 0.7)


def engineer_features(df):
    """
    Cria features derivadas a partir dos dados brutos.
    Mesma funcao definida em 2_model_pipeline.ipynb.
    """
    df = df.copy()

    # Total Exposto (denominador para ratios)
    if all(c in df.columns for c in ['valor_vencido', 'valor_por_vencer', 'valor_quitado']):
        df['total_exposto'] = df['valor_vencido'] + df['valor_por_vencer'] + df['valor_quitado']

    # Razao de Inadimplencia
    if 'valor_vencido' in df.columns and 'total_exposto' in df.columns:
        df['razao_inadimplencia'] = df['valor_vencido'] / (df['total_exposto'] + 1)

    # Taxa de Cobertura da Divida
    if 'valor_quitado' in df.columns and 'valor_total_pedido' in df.columns:
        df['taxa_cobertura_divida'] = df['valor_quitado'] / (df['valor_total_pedido'] + 1)

    # Razao Vencido vs Pedido
    if 'valor_vencido' in df.columns and 'valor_total_pedido' in df.columns:
        df['razao_vencido_pedido'] = df['valor_vencido'] / (df['valor_total_pedido'] + 1)

    # Flag de Risco Juridico (apenas quant_protestos - quant_acao_judicial excluida por >=99% zeros)
    if 'quant_protestos' in df.columns:
        df['flag_risco_juridico'] = (df['quant_protestos'] > 0).astype(int)

    # Ticket Medio de Protestos
    if 'valor_protestos' in df.columns and 'quant_protestos' in df.columns:
        df['ticket_medio_protestos'] = df['valor_protestos'] / (df['quant_protestos'] + 1)

    # Complexidade de Pagamento
    if 'forma_pagamento' in df.columns:
        df['qtd_parcelas'] = df['forma_pagamento'].apply(
            lambda x: str(x).count('/') + 1 if pd.notna(x) and str(x) not in ('nan', 'missing', 'unknown', '') else 1
        )

    return df


def to_categorical(x):
    """Converte valor para string categorica. Trata missing como 'unknown'."""
    if pd.isna(x) or x is None or x == 'missing' or x == '':
        return 'unknown'
    try:
        # Numerico -> int -> str (ex: 3.0 -> '3', para month)
        return str(int(float(x)))
    except (ValueError, TypeError):
        return str(x)  # String normal (ex: 'Boleto')


print(f"Modelo: {metadata['model_name']}")
print(f"ROC-AUC: {metadata['test_roc_auc']:.4f} | F1: {metadata['test_f1']:.4f}")
print(f"CV ROC-AUC: {metadata['cv_roc_auc_mean']:.4f} (+/- {metadata['cv_roc_auc_std']:.4f})")
print(f"\nThresholds: F1={THRESHOLD_F1:.4f} | F2={THRESHOLD_F2:.4f} | F0.5={THRESHOLD_F05:.4f}")
print(f"\nFeatures brutas ({len(RAW_FEATURES)}): usuario fornece")
print(f"Features derivadas ({len(DERIVED_FEATURES)}): calculadas por engineer_features()")
print(f"Total features pipeline ({len(ALL_FEATURES)}): {len(RAW_FEATURES)} brutas + {len(DERIVED_FEATURES)} derivadas")
print(f"\nFeatures categoricas: {CATEGORICAL_FEATURES}")

Modelo: XGBoost
ROC-AUC: 0.9251 | F1: 0.6709
CV ROC-AUC: 0.9197 (+/- 0.0021)

Thresholds: F1=0.6472 | F2=0.4619 | F0.5=0.7927

Features brutas (14): usuario fornece
Features derivadas (7): calculadas por engineer_features()
Total features pipeline (21): 14 brutas + 7 derivadas

Features categoricas: ['tipo_sociedade', 'atividade_principal', 'forma_pagamento', 'opcao_tributaria', 'month']


## Funcao de Predicao

- `threshold=None` usa o otimo F1 do metadata
- Aceita apenas features brutas (19) — 7 features derivadas sao calculadas automaticamente
- Features derivadas: `total_exposto`, `razao_inadimplencia`, `taxa_cobertura_divida`, `razao_vencido_pedido`, `flag_risco_juridico`, `ticket_medio_protestos`, `qtd_parcelas`
- Missing values: categoricas → `'unknown'`, numericas → `NaN` (imputer trata)
- O pipeline aplica encoding e scaling internamente
- Retorna: `{default, probability, confidence, threshold_used}`

In [3]:
def predict_default(input_dict, threshold=None):
    """
    Predicao de default com feature engineering automatico.

    Aceita features brutas (sem derivadas) - engineer_features() cria automaticamente.
    Missing values: categoricas -> 'unknown', numericas -> NaN (imputer usa mediana).
    Categoricas recebidas como numero (ex: month=3) sao convertidas para string ('3').
    """
    if threshold is None:
        threshold = THRESHOLD_F1
    try:
        input_df = pd.DataFrame([input_dict])

        # Tratar missing values e tipos
        for col in input_df.columns:
            if col in CATEGORICAL_FEATURES:
                # Categoricas: converter para string, missing -> 'unknown'
                input_df[col] = input_df[col].apply(to_categorical)
            else:
                # Numericas: 'missing' -> NaN, garantir tipo numerico
                input_df[col] = pd.to_numeric(
                    input_df[col].replace('missing', np.nan), errors='coerce'
                )

        # Filtrar valor_total_pedido negativo (inconsistencia - EDA)
        if 'valor_total_pedido' in input_df.columns:
            vtp = pd.to_numeric(input_df['valor_total_pedido'], errors='coerce')
            if (vtp < 0).any():
                return {"error": "valor_total_pedido negativo detectado - possivel inconsistencia nos dados"}

        # Feature engineering automatico (cria features derivadas)
        input_df = engineer_features(input_df)

        # Preencher features nao fornecidas
        for feature in ALL_FEATURES:
            if feature not in input_df.columns:
                input_df[feature] = 'unknown' if feature in CATEGORICAL_FEATURES else np.nan

        input_df = input_df[ALL_FEATURES]
        prob = float(model.predict_proba(input_df)[0, 1])
        pred = 1 if prob >= threshold else 0
        return {
            "default": pred,
            "probability": round(prob, 4),
            "confidence": round(max(prob, 1 - prob), 4),
            "threshold_used": round(threshold, 4)
        }
    except Exception as e:
        return {"error": str(e)}


def predict_default_batch(input_list, threshold=None):
    """Predicao em batch. Mesma interface, aceita lista de dicts."""
    return [predict_default(d, threshold) for d in input_list]

## Exemplos de Uso

Os exemplos usam apenas **features brutas** (19). As 7 features derivadas
(`total_exposto`, `razao_inadimplencia`, `taxa_cobertura_divida`, `razao_vencido_pedido`,
`flag_risco_juridico`, `ticket_medio_protestos`, `qtd_parcelas`)
sao calculadas automaticamente pela funcao `engineer_features()`.

Missing values em categoricas sao tratados como `'unknown'` (nao NaN).

In [4]:
# Exemplo 1: Cliente bom pagador (apenas features brutas)
# Features derivadas sao calculadas automaticamente por engineer_features()
cliente_bom = {
    # Numericas - historico
    "default_3months": 0,
    "ioi_36months": 95,
    "ioi_3months": 98,
    # Numericas - valores financeiros
    "valor_por_vencer": 30000,
    "valor_vencido": 0,
    "valor_quitado": 500000,
    "quant_protestos": 0,
    "valor_protestos": 0,
    "valor_total_pedido": 25000,
    # Categoricas (month incluido — correlacao com default no EDA)
    "month": 6,
    "tipo_sociedade": "Limitada",
    "atividade_principal": "Comercio varejista",
    "forma_pagamento": "Boleto",
    "opcao_tributaria": "simples nacional",
}

result = predict_default(cliente_bom)
print("=== Cliente Bom Pagador ===")
print(f"Default: {result['default']} | Prob: {result['probability']:.2%} | Confianca: {result['confidence']:.2%} | Threshold: {result['threshold_used']}")

=== Cliente Bom Pagador ===
Default: 0 | Prob: 28.49% | Confianca: 71.51% | Threshold: 0.6472


In [5]:
# Exemplo 2: Cliente alto risco (apenas features brutas)
# Features derivadas (razao_inadimplencia, flag_risco_juridico, etc.)
# sao calculadas automaticamente por engineer_features()
cliente_risco = {
    # Numericas - historico
    "default_3months": 8,
    "ioi_36months": 15,
    "ioi_3months": 4,
    # Numericas - valores financeiros
    "valor_por_vencer": 350000,
    "valor_vencido": 12000,
    "valor_quitado": 1800000,
    "quant_protestos": 5,
    "valor_protestos": 45000,
    "valor_total_pedido": 8500,
    # Categoricas
    "month": 11,
    "tipo_sociedade": "sociedade empresaria limitada",
    "atividade_principal": "loja de departamentos",
    "forma_pagamento": "30/60/90/120",
    "opcao_tributaria": "simples nacional",
}

result = predict_default(cliente_risco)
print("=== Cliente Alto Risco ===")
print(f"Default: {result['default']} | Prob: {result['probability']:.2%} | Confianca: {result['confidence']:.2%} | Threshold: {result['threshold_used']}")

=== Cliente Alto Risco ===
Default: 1 | Prob: 73.02% | Confianca: 73.02% | Threshold: 0.6472


## Comparacao de Thresholds

Thresholds carregados automaticamente do `model_metadata.json`.

In [6]:
# Comparar thresholds usando o cliente de risco como referencia
thresholds = [
    ("F2 (max recall)", THRESHOLD_F2),
    ("F1 (balanceado)", THRESHOLD_F1),
    ("F0.5 (max precision)", THRESHOLD_F05),
]

print("=== Comparacao de Thresholds ===")
print(f"Probabilidade do cliente: {predict_default(cliente_risco)['probability']:.4f}\n")

for label, t in thresholds:
    r = predict_default(cliente_risco, threshold=t)
    status = "DEFAULT" if r["default"] == 1 else "PAGO"
    print(f"  {label:25s} | threshold={t:.4f} | {status:7s} | prob={r['probability']:.4f}")


=== Comparacao de Thresholds ===
Probabilidade do cliente: 0.7302

  F2 (max recall)           | threshold=0.4619 | DEFAULT | prob=0.7302
  F1 (balanceado)           | threshold=0.6472 | DEFAULT | prob=0.7302
  F0.5 (max precision)      | threshold=0.7927 | PAGO    | prob=0.7302


## Predicao em Batch

In [7]:
# Batch com 3 perfis distintos
# 3o perfil: dados vazios — categoricas preenchidas com 'unknown', numericas com NaN
batch = [cliente_bom, cliente_risco, {}]
labels = ["Bom pagador", "Alto risco", "Dados vazios"]

results = predict_default_batch(batch)

print("=== Predicao em Batch ===")
for label, r in zip(labels, results):
    print(f"  {label:15s} | default={r['default']} | prob={r['probability']:.4f}")

=== Predicao em Batch ===
  Bom pagador     | default=0 | prob=0.2849
  Alto risco      | default=1 | prob=0.7302
  Dados vazios    | default=0 | prob=0.1294


## Validacao com Dados Reais

In [8]:
# Validacao com amostra do dataset original
df = pd.read_csv('../_data/dataset_2021-5-26-10-14.csv', sep='\t', encoding='utf-8')
# predict_default() trata missing values e tipos internamente:
# - Categoricas: 'missing'/NaN -> 'unknown', numerico -> str (ex: month 3 -> '3')
# - Numericas: 'missing'/NaN -> NaN (imputer usa mediana do treino)
# - Features derivadas: calculadas automaticamente por engineer_features()

np.random.seed(1234)
sample = df.sample(10)

acertos = 0
print("=== Validacao (10 registros aleatorios) ===")
for idx, row in sample.iterrows():
    real = int(row['default'])
    pred = predict_default(row.drop('default').to_dict())
    match = "OK" if real == pred["default"] else "ERRO"
    acertos += 1 if match == "OK" else 0
    print(f"  #{idx:5d} | real={real} pred={pred['default']} prob={pred['probability']:.4f} | {match}")

print(f"\nAcuracia na amostra: {acertos}/10 ({acertos*10}%)")

=== Validacao (10 registros aleatorios) ===
  #16196 | real=1 pred=0 prob=0.5734 | ERRO
  #86077 | real=0 pred=0 prob=0.1482 | OK
  #48921 | real=0 pred=0 prob=0.0806 | OK
  #48177 | real=0 pred=0 prob=0.0599 | OK
  #78842 | real=0 pred=0 prob=0.1697 | OK
  #103323 | real=1 pred=1 prob=0.9942 | OK
  # 5947 | real=0 pred=0 prob=0.5221 | OK
  #27990 | real=0 pred=0 prob=0.1360 | OK
  #22715 | real=1 pred=1 prob=0.9287 | OK
  #32477 | real=0 pred=0 prob=0.0939 | OK

Acuracia na amostra: 9/10 (90%)


## Exportar Script Standalone

In [9]:
script = '''
"""predict.py - Predicao de default (X-Health Credit Risk).

Feature engineering automatico: features derivadas calculadas internamente.
Missing values: categoricas -> 'unknown', numericas -> NaN (imputer usa mediana).
month e tratado como categorica (correlacao com default no EDA).

Decisoes de qualidade de dados (baseadas no EDA):
- valor_total_pedido negativo: rejeitado (possivel inconsistencia nos dados)
- 6 variaveis com >=99% zeros excluidas do modelo (possivel filtro pre-venda):
  participacao_falencia_valor, falencia_concordata_qtd, acao_judicial_valor,
  dividas_vencidas_qtd, dividas_vencidas_valor, quant_acao_judicial
"""
import pandas as pd, numpy as np, joblib, json
from pathlib import Path

model = joblib.load(list(Path('models/').glob('model_*.pkl'))[0])
with open('models/model_metadata.json') as f: metadata = json.load(f)

ALL_FEATURES = metadata['numeric_features'] + metadata['high_cardinality_features'] + metadata['low_cardinality_features']
CATEGORICAL_FEATURES = metadata['high_cardinality_features'] + metadata['low_cardinality_features']
DERIVED_FEATURES = metadata.get('derived_features', [])
THRESHOLD_F1 = metadata.get('optimal_threshold_f1', 0.5)


def engineer_features(df):
    df = df.copy()
    if all(c in df.columns for c in ['valor_vencido', 'valor_por_vencer', 'valor_quitado']):
        df['total_exposto'] = df['valor_vencido'] + df['valor_por_vencer'] + df['valor_quitado']
    if 'valor_vencido' in df.columns and 'total_exposto' in df.columns:
        df['razao_inadimplencia'] = df['valor_vencido'] / (df['total_exposto'] + 1)
    if 'valor_quitado' in df.columns and 'valor_total_pedido' in df.columns:
        df['taxa_cobertura_divida'] = df['valor_quitado'] / (df['valor_total_pedido'] + 1)
    if 'valor_vencido' in df.columns and 'valor_total_pedido' in df.columns:
        df['razao_vencido_pedido'] = df['valor_vencido'] / (df['valor_total_pedido'] + 1)
    # quant_acao_judicial excluida (>=99% zeros - possivel filtro pre-venda)
    if 'quant_protestos' in df.columns:
        df['flag_risco_juridico'] = (df['quant_protestos'] > 0).astype(int)
    if 'valor_protestos' in df.columns and 'quant_protestos' in df.columns:
        df['ticket_medio_protestos'] = df['valor_protestos'] / (df['quant_protestos'] + 1)
    if 'forma_pagamento' in df.columns:
        df['qtd_parcelas'] = df['forma_pagamento'].apply(
            lambda x: str(x).count('/') + 1 if pd.notna(x) and str(x) not in ('nan', 'missing', 'unknown', '') else 1
        )
    return df


def to_categorical(x):
    if pd.isna(x) or x is None or x == 'missing' or x == '':
        return 'unknown'
    try:
        return str(int(float(x)))
    except (ValueError, TypeError):
        return str(x)


def predict_default(input_dict, threshold=None):
    if threshold is None: threshold = THRESHOLD_F1
    df = pd.DataFrame([input_dict])
    for col in df.columns:
        if col in CATEGORICAL_FEATURES:
            df[col] = df[col].apply(to_categorical)
        else:
            df[col] = pd.to_numeric(df[col].replace('missing', np.nan), errors='coerce')
    # Validar valor_total_pedido negativo (inconsistencia nos dados - EDA)
    if 'valor_total_pedido' in df.columns:
        vtp = pd.to_numeric(df['valor_total_pedido'], errors='coerce')
        if (vtp < 0).any():
            return {"error": "valor_total_pedido negativo detectado - possivel inconsistencia nos dados"}
    df = engineer_features(df)
    for f in ALL_FEATURES:
        if f not in df.columns:
            df[f] = 'unknown' if f in CATEGORICAL_FEATURES else np.nan
    prob = float(model.predict_proba(df[ALL_FEATURES])[0, 1])
    return {"default": int(prob >= threshold), "probability": round(prob, 4), "threshold_used": round(threshold, 4)}

if __name__ == '__main__':
    print(predict_default({"ioi_3months": 3, "valor_vencido": 125000, "valor_total_pedido": 35000, "month": 6}))
'''

with open('../scripts/predict.py', 'w') as f:
    f.write(script)
print("Salvo em scripts/predict.py")

Salvo em scripts/predict.py
